In [13]:
import osmnx as ox
import geopandas as gpd
import pandas as pd

In [14]:
DEFAULT_CRS = "EPSG:4326"

In [15]:
ox.settings.requests_timeout=10000

In [16]:
# 1. Get the bounding polygon of São Paulo
city = "São Paulo, Brazil"
place = ox.geocoder.geocode_to_gdf(city)

In [17]:

tags_railway = {
 "railway": ["rail", "light_rail", "subway", "tram"],
} 
tags_highway = {
  "highway": ["motorway", "trunk", "primary"]
}
tags_aeroway = {
    "aeroway": ["aerodrome", "heliport", "runway", "taxiway", "apron"]
} 
tags_power = {
    "power": ["line", "minor_line", "tower", "pole", "plant", "generator"],
}
tags_landuse = {
 "landuse": ["industrial"],
}
tags_natural = {
    "natural": ["water", "bay", "strait"],
}

tags_waterway = {
    "waterway": ["river", "canal", "stream"]
}

In [18]:
gdf_railway = ox.geometries_from_polygon(place.geometry.iloc[0], tags_railway).reset_index()


/var/folders/kf/12t4yv8j7pg0z8zqbypmj9440000gn/T/ipykernel_24860/986735067.py:1: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf_railway = ox.geometries_from_polygon(place.geometry.iloc[0], tags_railway).reset_index()


In [19]:
gdf_highway = ox.geometries_from_polygon(place.geometry.iloc[0], tags_highway).reset_index()

/var/folders/kf/12t4yv8j7pg0z8zqbypmj9440000gn/T/ipykernel_24860/4280089468.py:1: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf_highway = ox.geometries_from_polygon(place.geometry.iloc[0], tags_highway).reset_index()


In [20]:
gdf_power = ox.geometries_from_polygon(place.geometry.iloc[0], tags_power).reset_index()

/var/folders/kf/12t4yv8j7pg0z8zqbypmj9440000gn/T/ipykernel_24860/1409962219.py:1: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf_power = ox.geometries_from_polygon(place.geometry.iloc[0], tags_power).reset_index()


In [30]:
gdf_aeroway = ox.geometries_from_polygon(place.geometry.iloc[0], tags_aeroway).reset_index()

/var/folders/kf/12t4yv8j7pg0z8zqbypmj9440000gn/T/ipykernel_24860/4059453001.py:1: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf_aeroway = ox.geometries_from_polygon(place.geometry.iloc[0], tags_aeroway).reset_index()


In [22]:
gdf_natural = ox.geometries_from_polygon(place.geometry.iloc[0], tags_natural).reset_index()

/var/folders/kf/12t4yv8j7pg0z8zqbypmj9440000gn/T/ipykernel_24860/2132015291.py:1: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf_natural = ox.geometries_from_polygon(place.geometry.iloc[0], tags_natural).reset_index()


In [23]:
gdf_waterway = ox.geometries_from_polygon(place.geometry.iloc[0], tags_waterway).reset_index()

/var/folders/kf/12t4yv8j7pg0z8zqbypmj9440000gn/T/ipykernel_24860/2135072112.py:1: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf_waterway = ox.geometries_from_polygon(place.geometry.iloc[0], tags_waterway).reset_index()


## Data Clean
- highways remove the element_type: relation
- gdf_aeroway remove element_type: node

In [26]:

hex_sp = gpd.read_file("../../data/shapeFiles/GRIDS_H3_SP_RES10") 


# Transform the CRS to EPSG:31983
hex_sp = hex_sp.to_crs(DEFAULT_CRS)
hex_sp.head()

,id_hex,geometry
0,8aa8100c3257fff,"POLYGON ((-46.63592 -23.53032, -46.63664 -23.5..."
1,8aa8100d2447fff,"POLYGON ((-46.53650 -23.53972, -46.53722 -23.5..."
2,8aa8100e1637fff,"POLYGON ((-46.66886 -23.59437, -46.66958 -23.5..."
3,8aa8100f0827fff,"POLYGON ((-46.59115 -23.61353, -46.59187 -23.6..."
4,8aa81010ec07fff,"POLYGON ((-46.65231 -23.83331, -46.65303 -23.8..."


# 2. Buffer each GeoDataFrame with 100 meters (convert to projected CRS for meters)


In [39]:
gdfs = [gdf_railway, gdf_highway, gdf_power, gdf_aeroway, gdf_natural, gdf_waterway]
buffered_gdfs = []

for gdf in gdfs:
    # Reproject to a projected CRS, apply 100m buffer, and revert to original CRS
    gdf_projected = gdf.to_crs(epsg=3857)
    gdf_buffered = gdf_projected.buffer(100).to_crs(epsg=4326)
    buffered_gdfs.append(gdf_buffered)


In [40]:
# 3. Perform intersections with each buffered GeoDataFrame and add boolean columns
hex_sp["railway_intersection"] = hex_sp.intersects(buffered_gdfs[0].unary_union)
hex_sp["highway_intersection"] = hex_sp.intersects(buffered_gdfs[1].unary_union)
hex_sp["power_intersection"] = hex_sp.intersects(buffered_gdfs[2].unary_union)
hex_sp["aeroway_intersection"] = hex_sp.intersects(buffered_gdfs[3].unary_union)
hex_sp["natural_intersection"] = hex_sp.intersects(buffered_gdfs[4].unary_union)
hex_sp["waterway_intersection"] = hex_sp.intersects(buffered_gdfs[5].unary_union)

In [41]:
hex_sp.head()

,id_hex,geometry,railway_intersection,highway_intersection,power_intersection,aeroway_intersection,natural_intersection,waterway_intersection
0,8aa8100c3257fff,"POLYGON ((-46.63592 -23.53032, -46.63664 -23.5...",False,False,False,False,False,False
1,8aa8100d2447fff,"POLYGON ((-46.53650 -23.53972, -46.53722 -23.5...",False,False,False,False,False,False
2,8aa8100e1637fff,"POLYGON ((-46.66886 -23.59437, -46.66958 -23.5...",False,False,False,False,False,False
3,8aa8100f0827fff,"POLYGON ((-46.59115 -23.61353, -46.59187 -23.6...",False,False,False,False,False,False
4,8aa81010ec07fff,"POLYGON ((-46.65231 -23.83331, -46.65303 -23.8...",False,False,False,False,False,False


In [42]:
hex_sp[hex_sp['id_hex'] == '8aa810150937fff']

,id_hex,geometry,railway_intersection,highway_intersection,power_intersection,aeroway_intersection,natural_intersection,waterway_intersection
87189,8aa810150937fff,"POLYGON ((-46.72339 -23.78188, -46.72411 -23.7...",False,True,False,False,False,True


In [44]:
hex_sp.to_file("../../data/generated/WGS84_hex_sp_osm.gpkg", driver="GPKG")
